In [1]:
import arcpy

In [2]:
arcpy.env.workspace = '..\\scratch'
arcpy.env.scratchWorkspace = '..\\scratch\\scratch.gdb'
arcpy.env.overwriteOutput = True

In [3]:
lcp_fc = 'LCPNetwork.shp'
bgprj_fc = r'C:\Workspace\Gits\BioGas_optimization\OptimizeViewer\OptimizeViewer.gdb\Biogas_prj'

In [4]:
#Convert vertices to points
lcp_ends = arcpy.management.FeatureVerticesToPoints(in_features=lcp_fc,
                                                    out_feature_class='LCP_ends',
                                                    point_location='BOTH_ENDS').getOutput(0)

In [12]:
#Dissolve the points
lcp_ends_dslv = arcpy.management.Dissolve(lcp_ends, 
                                          "LCP_ends_Dissolve", 
                                          None, 
                                          None, 
                                          "SINGLE_PART", 
                                          ).getOutput(0)

In [13]:
#Add a unique ID field to the lcp_split
arcpy.management.AddField(in_table=lcp_ends_dslv,
                          field_name='PtID',
                          field_type='SHORT')

<Result '..\\scratch\\LCP_ends_Dissolve.shp'>

In [15]:
arcpy.management.CalculateField(in_table=lcp_ends_dslv,
                                field='PtID',
                                expression='!FID!',
                                expression_type='PYTHON3'
                               )

<Result '..\\scratch\\LCP_ends_Dissolve.shp'>

In [25]:
#Spatially join the BG source ID to the Pts
lcp_ends = arcpy.analysis.SpatialJoin(target_features=lcp_ends_dslv, 
                           join_features=bgprj_fc, 
                           out_feature_class='LCP_ends2', 
                           join_operation="JOIN_ONE_TO_ONE", 
                           join_type="KEEP_ALL", 
                           field_mapping=('Id "Id" true true false 6 Long 0 6,First,#,LCP_ends_Dissolve,Id,-1,-1;'+
                                          'PtID "PtID" true true false 5 Long 0 5,First,#,LCP_ends_Dissolve,PtID,-1,-1;'+
                                          'Total_Potential_Methane_Yield_1 "Total Potential Methane Yield (scf/h)" true true false 8 Double 0 0,First,#,Biogas_Prj,Total_Potential_Methane_Yield_1,-1,-1'), 
                           match_option="WITHIN_A_DISTANCE", 
                           search_radius="500 Meters").getOutput(0)

In [27]:
#Split the LCP at the vertices
lcp_split = arcpy.management.SplitLineAtPoint(in_features=lcp_fc,
                                              point_features=lcp_ends_dslv,
                                              out_feature_class='LCP_split',
                                              search_radius='500 meters').getOutput(0)

In [29]:
#Spatially join the vertices to the lines
lcp_edges = arcpy.analysis.SpatialJoin(target_features=lcp_split,
                                       join_features=lcp_ends_dslv,
                                       out_feature_class='lcp_join',
                                       join_operation='JOIN_ONE_TO_MANY',
                                       match_option='INTERSECT',
                                       search_radius='500 meters'
                                      ).getOutput(0)

In [10]:
#Spatially join the biogas source to the lines


'..\\scratch\\lcp_join.shp'